<a href="https://colab.research.google.com/github/tarcisiolucas/Analise-dados-sensores/blob/main/An%C3%A1lise_dados_sensores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo para análise de dados de sensores

Essa parte inicial do notebook tem como objetivo demonstrar um **modelo de análise de dados de um sensor**, abrangendo os principais requisitos para o entendimento e abstração de informações com base nos dados. Sendo os requisitos:


1. Avaliação e inclusão das bibliotecas necessárias para realizar as operações
2.   Leitura do arquivo disponibilizado pela Datalog
3.   Permitir filtrar ou realizar um resample dos dados 
4. Separação dos dados necessários para essa análise (dois ou mais?)
5. Realizar uma visualização interativa dos dados filtrados ou até mesmo deles brutos
6. Permitir operações (soma, integração e derivação) entre graficos distintos de mais de um dado

Após a definição do modelo, serão criadas novas seções **nesse mesmo caderno** que utilizam ele como base para analisar os dados de um ou mais sensores

### Sugestões:
* Estude pipelines para facilitar o tratamento dos dados
* Não economize elementos gráficos para melhorar a visualização de seus dados
* Seus dados precisam contar uma história e provar algo (Ex. testes de sensores provam que o sensor é confiável. Seus dados devem mostrar isso)
* Certifique-se da origem do seu dados. Lembre-se do GIGO (Garbage In, Garbage Out). Não adianta ter um modelo eficiente se os dados não foram aquisitados da forma correta
* Sempre pesquise novas bibliotecas e frameworks na área de Data Science
* Crie uma conta no Kaggle. Existem milhares de projetos nos quais você pode se inspirar para otimizar seu modelo. No YouTube existem vários projetos também, muitos deles criados na hora em formato de curso

##1. Avaliação e inclusão das bibliotecas necessárias para realizar as operações

Como as operações serão majoritariamente realizadas utilizando leitura de arquivos e plot de gráficos utilizaremos as seguintes bibliotecas:

*   **pandas** - operações com dataframes
*   **matplotlib** - visualização gráfica dos dados
*   **seaborn** - estatística e visualização gráfica mais otimizada. É baseada no matplotlib
*   **plotly** - plotar gráficos interativos
* **numpy** - Cálculo dos gráficos

In [59]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.graph_objs as go

##2. Leitura e resumo do arquivo disponibilizado pela Datalog

### Para realizar a leitura, primeiramente é necessário um repositório no GitHub contendo o arquivo.

###-> Para incluir i dataset a partir do repositório no Github:

* Entre no repositório

https://github.com/caiussouza/Controls-Data-Analysis
* Clique no arquivo a ser lido. No exemplo, é o "primeira_analise.csv"
* Clique em "Raw" e copie a url. No exemplo, é

https://raw.githubusercontent.com/caiussouza/Controls-Data-Analysis/main/primeira_analise.csv
* Utilize o método .read_csv('INSIRA URL')

*nota: O arquivo disponibilizado pela datalog utiliza tabulação como metodo de separação dos intens e não espaço como aparenta e nem vírgula que é o padrão de arquivos csv, portanto na hora de realizar a leitura passar `sep = '\t'` como parâmetro*

In [44]:
url_dataframe = 'https://raw.githubusercontent.com/tarcisiolucas/Analise-dados-sensores/main/09_30_Test_Data/ARQ19.txt'
teste = pd.read_csv(url_dataframe, sep='\t')

###Visualizando o resumo do dataset

Exemplos:

*   **teste.info()** - exibe informações sobre o dataframe, incluindo o número de linhas e colunas, nomes e tipos de dados das colunas e o uso de memória.
*   **teste.head()** - exibe as 5 primeiras linhas
*   **teste.describe()** - exibe informações estatísticas sobre o dataframe, como médias, desvios padrões

In [ ]:
teste.info()

In [ ]:
teste.head()

In [ ]:
teste.describe()

##3. Permitir filtrar ou realizar um resample dos dados



##4. Separação dos dados necessários para essa análise

Agora que os dados provavelmente já estão filtrados, vamos separar os que serão estudados do volume imenso distribuído pela datalogger. Para isso criaremos uma variavel que recebe os dados que queremos e iremos definir o tempo como índice

In [ ]:
#troque o 'data_1' 'data_2' ou 'data_3' para o dado a ser estudado,
#também podem ser adicionados mais dados caso necessário

data_1 = 'ACCEL_X'
data_2 = 'ACCEL_Y'
data_3 = 'ACCEL_Z'

data_analisys = teste[['TIMER', data_1, data_2, data_3]]
data_analisys.set_index('TIMER')

Como os dados analisados tem um offset de 10000 e valor dado em g, iremos subtrair o valor do offset e converter para m/s

**Avalie sempre que necessário o dado a ser estudado para verificar a necessidade de conversão**

In [56]:
g = 9.8
data_analisys_converted = pd.DataFrame()
data_analisys_converted[[data_1,data_2,data_3]] = data_analisys[[data_1, data_2, data_3]].apply(lambda x: (x - 10000)*(g/1000))

##5. Visualização interativa dos dados filtrados ou brutos

Com os dados em mãos, agora iremos gerar gráficos de suas alterações em função do tempo. O meio utilizado é a biblioteca plotly

In [ ]:
#declarando o eixo x como o índice da tabela (definimos anteriormente como TIMER)
x = data_analisys.TIMER

y_in_data_1 = data_analisys_converted.ACCEL_X
y_in_data_2 = data_analisys_converted.ACCEL_Y
y_in_data_3 = data_analisys_converted.ACCEL_Z

# Criar os traços do gráfico
trace1 = go.Scatter(x=x, y=y_in_data_1, name='Accelerometer in x')
trace2 = go.Scatter(x=x, y=y_in_data_2, name='Accelerometer in y')
trace3 = go.Scatter(x=x, y=y_in_data_3, name='Accelerometer in z')

# Criar a figura e configurar o layout
fig = go.Figure(data=[trace1, trace2, trace3])
fig.update_layout(title='Accelerometer Data',
                  xaxis_title='Time (ms)',
                  yaxis_title='acceleration (m/s^2)')

# Exibir o gráfico interativo em um navegador da web
fig.show()



##6. Operações (soma, integração e derivação) entre graficos distintos de mais de um dado

Agora que já podemos vizualizar os dados, faremos operações com esses gráficos com o intuito de obter informações relevantes para a análise. As operações que inicialmente serão feitas são:

* Derivar
* Intergrar
* Soma e Subtração de dois dados diferentes

### Derivar um gráfico

Estamos usando a função np.gradient() do NumPy para calcular a derivada dos dados em cada uma das colunas.

In [ ]:
# calcular a derivada dos dados
dydx_in_data_1 = np.gradient(y_in_data_1)
dydx_in_data_2 = np.gradient(y_in_data_2)
dydx_in_data_3 = np.gradient(y_in_data_3)

# Criar os traços do gráfico
trace1 = go.Scatter(x=x, y=dydx_in_data_1, name='Accelerometer in x (derivative)')
trace2 = go.Scatter(x=x, y=dydx_in_data_2, name='Accelerometer in y (derivative)')
trace3 = go.Scatter(x=x, y=dydx_in_data_3, name='Accelerometer in z (derivative)')

# Criar a figura e configurar o layout
fig = go.Figure(data=[trace1, trace2, trace3])
fig.update_layout(title='Derivative of Accelerometer Data',
                  xaxis_title='Time (ms)',
                  yaxis_title='acceleration (m/s^2)')

# Exibir o gráfico interativo em um navegador da web
fig.show()

### Integrar um gráfico

Estamos usando a função np.cumsum() do NumPy para calcular a integral dos dados em cada uma das colunas. Em seguida, estamos criando um novo gráfico com os dados integrados.

*nota: aparentemente não deu muito certo. Avaliar outra forma de realizar depois*

In [ ]:
# calcular a integral dos dados
integral_in_data_1 = np.cumsum(y_in_data_1) * (x[1]-x[0])
integral_in_data_2 = np.cumsum(y_in_data_2) * (x[1]-x[0])
integral_in_data_3 = np.cumsum(y_in_data_3) * (x[1]-x[0])

# Criar os traços do gráfico
trace1 = go.Scatter(x=x, y=integral_in_data_1, name='Accelerometer in x (integral)')
trace2 = go.Scatter(x=x, y=integral_in_data_2, name='Accelerometer in y (integral)')
trace3 = go.Scatter(x=x, y=integral_in_data_3, name='Accelerometer in z (integral)')

# Criar a figura e configurar o layout
fig = go.Figure(data=[trace1, trace2, trace3])
fig.update_layout(title='Integral of Accelerometer Data',
                  xaxis_title='Time (ms)',
                  yaxis_title='velocity (m/s)')

# Exibir o gráfico interativo em um navegador da web
fig.show()

### Soma e subtração

aqui será somado e subtraído dois dados apenas como exemplo.

In [ ]:
y_in_data_1 = data_analisys_converted.ACCEL_X
y_in_data_2 = data_analisys_converted.ACCEL_Y

sum_data = y_in_data_1 + y_in_data_2
sub_data = y_in_data_1 - y_in_data_2

# Criar os traços do gráfico
trace1 = go.Scatter(x=x, y=sum_data, name='Sum data_1 + data_2')
trace2 = go.Scatter(x=x, y=sub_data, name='Sub data_1 - data_2')

# Criar a figura e configurar o layout
fig = go.Figure(data=[trace1, trace2])
fig.update_layout(title='Accelerometer Data',
                  xaxis_title='Time (ms)',
                  yaxis_title='acceleration (m/s^2)')

# Exibir o gráfico interativo em um navegador da web
fig.show()